### Silver Layer: Cleansing and Transformation
normalize data in the Silver tier. Use partitioning to improve read and write performance.

### Camada Silver: Limpeza e Transformação
normalizar os dados na camada Silver. Use particionamento para melhorar o desempenho de leitura e escrita (PT-BR)

In [0]:
# Import the required libraries
# Importar as bibliotecas necessárias

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Transformação Data Silver") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Sets a fixed number of partitions to shuffle, improving parallelism
# Sets the maximum partition size to avoid too many small files
# Uses the Snappy codec for fast compression, optimizing read and write times
# Enables adaptive optimizations, adjusting the number of partitions dynamically based on data size


# Define um número fixo de partições para shuffle, melhorando o paralelismo                 
# Define o tamanho máximo de partições para evitar muitos arquivos pequenos        
# Usa o codec Snappy para compressão rápida, otimizando tempo de leitura e escrita    
# Habilita otimizações adaptativas, ajustando o número de partições dinamicamente com base no tamanho dos dados

# Definir caminhos de armazenamento no Data Lake
# Ler dados na Bronze e Armazenar Silver (PT-BR)

bronze_path = "/mnt/lhdw/layer_bronze/detran2025"
silver_path = "/mnt/lhdw/layer_silver/detran2025"


###Read data from Bronze layer for transformation into Silver layer
###Ler o dados da camada Bronze para transformação na camada Silver (PT-BR)

In [0]:
# Read data from Bronze layer
# Ler dados da camada Bronze (PT-BR)

df_bronze = spark.read.format("parquet").load(bronze_path)

## Adding the silver_ingestion column for data control and traceability
## Adicionando a coluna silver_ingestion para controle e rastreabilidade dos dados (PT-BR)

In [0]:
df_bronze = df_bronze.withColumn('silver_ingestion' , current_timestamp())

## creating a temp view for data processing
## criando um temp view para tratamento de dados

In [0]:
df_bronze.createOrReplaceTempView("detran")

### Data Cleaning
Data cleaning is a crucial process for ensuring data quality. It involves removing duplicate or incorrect data, standardizing data formats and values, and enriching data with additional information. In addition, it is important to check for and correct quality issues such as errors and inconsistencies to ensure that the data is accurate and reliable.

### Limpeza dos dados
A limpeza de dados é um processo crucial para garantir a qualidade dos dados. Isso envolve a remoção de dados duplicados ou incorretos, a padronização de formatos e valores de dados e o enriquecimento de dados com informações adicionais. Além disso, é importante verificar e corrigir problemas de qualidade, como erros e inconsistências, para garantir que os dados sejam precisos e confiáveis. (PT-BR)

In [0]:
df_silver = spark.sql("""
SELECT DISTINCT
  id AS ID,
  data_inversa AS DATA_OCORRENCIA,
  upper(ANO) AS ANO,
  UPPER(Mes) AS MES, 
  SPLIT(DATA_INVERSA, '-')[2] AS DIA,
  UPPER(dia_semana) AS DIA_SEMANA,
  horario AS HORARIO,
  uf AS UF,
  br AS NUM_BR,
  km AS KM,
  municipio AS MUNICIPIO,
  UPPER(causa_acidente) AS CAUSA_ACIDENTE,
  UPPER(tipo_acidente) AS TIPO_ACIDENTE,
  UPPER(classificacao_acidente) AS CLASSIFICACAO_ACIDENTE,
  UPPER(fase_dia) AS FASE_DIA,
  UPPER(sentido_via) AS SENTIDO_VIA,
  UPPER(condicao_metereologica) AS CONDICAO_METEREOLOGICA,
  UPPER(tipo_pista) AS TIPO_PISTA,
  UPPER(REGEXP_REPLACE(tracado_via, ';', '-')) AS TRACADO_VIA,
  UPPER(uso_solo) AS USOU_SOLO,
  pessoas AS QTD_PESSOAS,
  mortos AS QTD_MORTOS,
  feridos_leves AS QTD_FERIDOS_LEVES,
  feridos_graves AS QTD_FERIDOS_GRAVES,
  ilesos AS QTD_ILESOS,
  ignorados AS QTD_IGNORADOS,
  feridos AS QTD_FERIDOS,
  veiculos AS QTD_VEICULOS,
  SPLIT(regional, '-')[0] AS REGIONAL,
  SPLIT(regional, '-')[1] AS UF_REGIONAL,
  SPLIT(DELEGACIA, '-')[0] AS DELEGACIA,
  SPLIT(delegacia, '-')[1] AS UF_DELEGACIA,
  SPLIT(UOP, '-')[0] AS UOP, 
  SPLIT(UOP, '-')[2] AS UF_UOP,
  latitude AS LATITUDE,
  longitude AS LONGITUDE,
  bronze_ingestion AS BRONZE_INGESTION,
  silver_ingestion AS SILVER_INGESTION
FROM detran
""")


### Write Silver Transformations
Partitioning by year and month to optimize date-based queries, with recommended file size in Parquet format

### Gravar transformações Silver
Particionamento por ano e mês para otimizar consultas baseadas em data, com recomendação de tamanho de arquivo em formato Parquet (PT-BR)

In [0]:
# Salva particionado por Ano e Mes
df_silver.write \
    .option("maxRecordsPerFile", 50000) \
    .partitionBy("Ano", "Mes") \
    .format("parquet") \
    .mode("overwrite") \
    .save(silver_path)


## Evidence of data in the Silver layer
## Evidência dos dados na camada Silver (PT-BR).

In [0]:
%fs ls /mnt/lhdw/layer_silver/detran2025

path,name,size,modificationTime
dbfs:/mnt/lhdw/layer_silver/detran2025/Ano=2025/,Ano=2025/,0,0
dbfs:/mnt/lhdw/layer_silver/detran2025/_SUCCESS,_SUCCESS,0,1747791255000
dbfs:/mnt/lhdw/layer_silver/detran2025/_committed_7547434564006332486,_committed_7547434564006332486,35,1747542432000


In [0]:
#Record count
#Contagem de registros

df_silver.count()

Out[7]: 16715

%md

**Justification for partitioning:**

partitionBy("Year", "Month"): Partitioning data by the Year and Month columns helps optimize reading when we want to filter or query data based on specific periods. This reduces the number of files scanned in queries, improving performance.

**Justificativa para particionamento:**

partitionBy("Ano", "Mes"): Particionar os dados pelas coluna Ano e Mês ajuda a otimizar a leitura quando queremos filtrar ou consultar dados baseados em periodos específicos. Isso reduz o número de arquivos escaneados em consultas, melhorando a performance (PT-BR).



### Clearing Memory
### Limpando a Memória (PT-BR)

In [0]:
import gc
gc.collect()

Out[8]: 285

In [0]:
del df_bronze
del df_silver